In [15]:
# data manipulations
import numpy as np
import pandas as pd

# data preaparation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# for the models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Prepare train and test data

In [5]:
df = pd.read_csv("sample.csv", index_col="filename")
df.head()

,rms,zcr,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,label
filename,,,,,,,,,,,,,,,,,,,,,
./Labelled/Bleatings/evt_001_003_034909_210502_134518.wav,0.006984,0.187475,-377.00934,95.880670,-42.657234,77.715480,-33.887910,33.206450,-2.512250,8.490808,...,-3.735909,8.510938,-6.560940,11.606267,-5.194788,7.343347,-0.619158,3.602003,0.078233,Bleatings
./Labelled/Bleatings/evt_000_001_002632_210416_073852.wav,0.062214,0.099904,-188.76020,145.959560,-68.584650,54.143364,-14.203813,1.955194,-5.561017,16.223532,...,2.527336,7.075812,-8.010324,9.080058,-7.769785,3.599059,5.259237,3.122564,0.849308,Bleatings
./Labelled/Others/evt_003_002_003569_210501_135115.wav,0.005709,0.127847,-474.72763,147.379090,-34.655480,61.227776,-11.979817,35.392506,0.455712,10.564780,...,7.283661,6.594476,0.378431,12.473774,-4.178606,12.500201,-5.843817,6.315655,6.741754,Others
./Labelled/Others/evt_000_001_020444_210423_220448.wav,0.020925,0.108683,-299.69928,171.500210,-83.280010,26.681047,-20.505777,20.649090,-2.309497,8.671416,...,-0.587268,7.369606,-3.986548,0.528011,-12.755464,4.904510,-4.969002,1.672815,3.406864,Others
./Labelled/Others/evt_001_002_018572_210425_094233.wav,0.004345,0.096446,-426.04117,123.996414,-34.002785,61.306442,-18.612503,27.970829,-2.241459,13.966869,...,-1.264318,10.060466,-7.955052,12.376381,-4.092283,5.181476,-1.892008,0.991476,-1.825804,Others


Extract and prepare the labels

In [7]:
label_list = df["label"]
encoder = LabelEncoder()
y = encoder.fit_transform(label_list)#Scaling the Feature columns
y

array([0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 0, 1, 1, 2, 2, 0,
       1, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 1, 2, 0, 0, 2, 0, 2, 0, 2,
       2, 0, 2, 2, 0, 0])

Extract and scale the data

In [9]:
df.iloc[:, :-1].head()

,rms,zcr,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,...,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19
filename,,,,,,,,,,,,,,,,,,,,,
./Labelled/Bleatings/evt_001_003_034909_210502_134518.wav,0.006984,0.187475,-377.00934,95.880670,-42.657234,77.715480,-33.887910,33.206450,-2.512250,8.490808,...,4.874284,-3.735909,8.510938,-6.560940,11.606267,-5.194788,7.343347,-0.619158,3.602003,0.078233
./Labelled/Bleatings/evt_000_001_002632_210416_073852.wav,0.062214,0.099904,-188.76020,145.959560,-68.584650,54.143364,-14.203813,1.955194,-5.561017,16.223532,...,6.356914,2.527336,7.075812,-8.010324,9.080058,-7.769785,3.599059,5.259237,3.122564,0.849308
./Labelled/Others/evt_003_002_003569_210501_135115.wav,0.005709,0.127847,-474.72763,147.379090,-34.655480,61.227776,-11.979817,35.392506,0.455712,10.564780,...,5.295946,7.283661,6.594476,0.378431,12.473774,-4.178606,12.500201,-5.843817,6.315655,6.741754
./Labelled/Others/evt_000_001_020444_210423_220448.wav,0.020925,0.108683,-299.69928,171.500210,-83.280010,26.681047,-20.505777,20.649090,-2.309497,8.671416,...,0.527445,-0.587268,7.369606,-3.986548,0.528011,-12.755464,4.904510,-4.969002,1.672815,3.406864
./Labelled/Others/evt_001_002_018572_210425_094233.wav,0.004345,0.096446,-426.04117,123.996414,-34.002785,61.306442,-18.612503,27.970829,-2.241459,13.966869,...,1.993648,-1.264318,10.060466,-7.955052,12.376381,-4.092283,5.181476,-1.892008,0.991476,-1.825804


In [12]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(df.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X

array([[-0.73413191,  3.77846783, -0.50206106, ...,  0.46472857,
         0.07726453, -0.18009678],
       [ 2.37351326,  0.37938022,  1.53377913, ...,  2.47569833,
        -0.1092453 ,  0.18887619],
       [-0.80589256,  1.4640087 , -1.55884582, ..., -1.32260144,
         1.13292101,  3.00851281],
       ...,
       [-0.2501768 , -0.08161662,  0.36693445, ..., -0.75709079,
         3.030069  , -0.23293907],
       [ 0.86756598,  0.6856358 ,  1.01886321, ...,  0.3091691 ,
        -1.31600965,  0.74727783],
       [-0.75060634, -0.20748089, -0.87869765, ...,  0.21256751,
        -1.14061103, -0.24615213]])

And finally split in train and test data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a model

Let's build a simple ANN 

In [34]:
model = keras.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
classifier = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=10)

Epoch 1/100
4/4 [==============================] - 0s 1ms/step - loss: 1.0166 - accuracy: 0.5000
Epoch 2/100
4/4 [==============================] - 0s 1ms/step - loss: 0.9798 - accuracy: 0.5500
Epoch 3/100
4/4 [==============================] - 0s 751us/step - loss: 0.9519 - accuracy: 0.5500
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9229 - accuracy: 0.5500
Epoch 5/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8983 - accuracy: 0.5500
Epoch 6/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8706 - accuracy: 0.6000
Epoch 7/100
4/4 [==============================] - 0s 1000us/step - loss: 0.8437 - accuracy: 0.6750
Epoch 8/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8151 - accuracy: 0.7250
Epoch 9/100
4/4 [==============================] - 0s 1ms/step - loss: 0.7867 - accuracy: 0.7750
Epoch 10/100
4/4 [==============================] - 0s 1ms/step - loss: 0.7611 - accuracy: 0.8250
Epoch 11/100
4/4 [======

In [36]:
results = model.evaluate(X_test, y_test, batch_size=10)
print("test loss, test acc:", results)

1/1 [==============================] - 0s 2ms/step - loss: 2.6659 - accuracy: 0.4000
test loss, test acc: [2.6658830642700195, 0.4000000059604645]


In [63]:
print("Generate predictions for 10 samples")
predictions = model.predict(X_test[:10])
predictions

Generate predictions for 10 samples


array([[9.9538273e-01, 2.3118521e-04, 4.3861605e-03],
       [4.6212319e-01, 3.1923538e-01, 2.1864146e-01],
       [9.9774534e-01, 2.6924635e-04, 1.9853825e-03],
       [4.3544129e-02, 1.4007849e-01, 8.1637734e-01],
       [1.8158157e-01, 4.0928763e-01, 4.0913078e-01],
       [9.9967170e-01, 4.2199659e-05, 2.8612255e-04],
       [9.9820757e-01, 3.2581971e-04, 1.4665831e-03],
       [3.7487142e-04, 1.3524566e-04, 9.9948990e-01],
       [9.0188587e-01, 5.9736811e-02, 3.8377404e-02],
       [7.4008860e-02, 2.6199090e-01, 6.6400027e-01]], dtype=float32)

In [65]:
y_test[:10]

array([0, 1, 2, 1, 2, 0, 1, 0, 0, 2])

In [64]:
real = y_test[:10]
predicted = np.argmax(predictions, axis=1)
pd.DataFrame(
    {"real": real,
     "predicted": predicted
    })

,real,predicted
0,0,0
1,1,0
2,2,0
3,1,2
4,2,1
5,0,0
6,1,0
7,0,2
8,0,0
9,2,2
